## Machine Learning

### Importación de las librerías necesarias

In [2]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

Se utilizarán las librerías sklearn.feature_extraction.text y sklearn.metrics.pairwise para implementar un sistema de recomendación basado en filtrado de contenido en el que se compararán las similitudes entre diferentes elementos de un conjunto de datos. La librería CountVectorizer se utilizará para convertir los datos de texto en una matriz numérica, mientras que cosine_similarity se utilizará para calcular la similitud coseno entre los elementos de la matriz. Esto permitirá generar recomendaciones personalizadas basadas en los patrones de similitud de los datos de entrada.

### Carga de datos

In [3]:
df=pd.read_csv(r"https://raw.githubusercontent.com/JersonGB22/ProyectoIndividual1_Henry/main/Datasets/ML_movie_transformation.csv")
df

,title,label
0,toy story,toy stori anim comedi famili led woodi andi to...
1,jumanji,jumanji adventur fantasi famili sibl judi pete...
2,grumpier old men,grumpier old men romanc comedi famili wed reig...
3,waiting to exhale,wait exhal comedi drama romanc cheat mistreat ...
4,father of the bride part ii,father bride part ii comedi just georg bank re...
...,...,...
21801,the sublet,sublet mysteri drama thriller horror sublet su...
21802,cadet kelly,cadet kelli comedi hyperact teenag kelli enrol...
21803,pooh's heffalump halloween movie,pooh heffalump halloween movi anim famili hall...
21804,mom,mom crime drama thriller bliss biolog teacher ...


### Matriz de Caracaterísticas

In [4]:
# Visualización del registro con mayor cantidad de palabras
word=df.label.str.split().str.len()
print("Número de Palabras: ",word.max())
df.label.iloc[word.idxmax()]

Número de Palabras:  160


'la vérité si je men 3 comedi eddi dov yvan et le autr no chaleureux ami ont migré du sentier moribond à la banlieu florissant daubervilli là où le vieux entrepreneur juif ont laissé le terrain à de jeun grossist chinoi courageux et dynamiqu la petit band est toujour aussi soudé solidair que lor de épisod précédent et la vie suit son cour au gré de petit évènement familiaux et de affair dov sembl toujour frivol eddi entrepren yvan transi karin désinvolt sandra résolu chochana naïv serg irrespons et mythoman quant à patrick il est amoureux et lheureus élue est loin dêtre facil daccè tout irait pour le mieux jusquà ce quun vent mauvai apport son lot dadversité compromett sérieusement la cohésion du group succomberontil sou lorag à la zizani ou bien une foi de plu à forc dentraid de ruse et dhabileté triompherontil de la crise avec panach le film manuel munz télégraph vertigo la vérité product franc françai'

In [5]:
# Visualización del registro con menor cantidad de palabras
print("Número de Palabras: ",word.min())
df.label.iloc[word.idxmin()]

Número de Palabras:  3


'four day napl'

In [6]:
cv=CountVectorizer(max_features=300,stop_words="english")
vector=cv.fit_transform(df.label).toarray()

* Al utilizar max_features=300, estoy seleccionando un número cercano al doble del máximo de palabras presentes en mis registros, que es de 160. Esta elección me permite capturar un rango más amplio de palabras relevantes en mis datos sin agregar un exceso de características irrelevantes. Al limitar el número de características a 300, puedo mantener un equilibrio entre la representatividad de las palabras seleccionadas y la eficiencia del modelo. Además, al eliminar las palabras comunes en inglés con stop_words="english", estoy asegurando que las características restantes sean más significativas y relevantes para mi modelo, lo que puede mejorar la calidad de las predicciones y reducir el ruido en los resultados obtenidos.

In [7]:
# Visualizando el array vector
print(vector.shape)
vector

(21806, 300)


array([[0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]], dtype=int64)

### Matriz de Similitud

In [8]:
similarity_score=cosine_similarity(vector)

In [9]:
# Visualizando el array similarity_score
print(similarity_score.shape)
similarity_score

(21806, 21806)


array([[1.        , 0.26490647, 0.32444284, ..., 0.31494489, 0.06131393,
        0.35320863],
       [0.26490647, 1.        , 0.22680461, ..., 0.22645541, 0.15430335,
        0.25925926],
       [0.32444284, 0.22680461, 1.        , ..., 0.2773501 , 0.06299408,
        0.27216553],
       ...,
       [0.31494489, 0.22645541, 0.2773501 , ..., 1.        , 0.10482848,
        0.26419797],
       [0.06131393, 0.15430335, 0.06299408, ..., 0.10482848, 1.        ,
        0.1028689 ],
       [0.35320863, 0.25925926, 0.27216553, ..., 0.26419797, 0.1028689 ,
        1.        ]])

* Crearemos una función que se utilizará para indexar las películas más similares a una película específica en el conjunto de datos. Dado un título de película, la función encuentra su índice correspondiente en el conjunto de datos y utiliza la matriz de similitud previamente calculada, para determinar los 5 elementos más similares a la película de referencia. La función devuelve una lista de índices correspondientes a las películas más similares en orden descendente, lo que permite utilizar esta información para hacer recomendaciones personalizadas en el endpoint 7 de la API de manera más rápida, eficiente y efectiva.

In [11]:
def index_movie(titulo:str):
    index=df[df.title==titulo].index[0]
    score=list(enumerate(similarity_score[index]))
    score=sorted(score,key=lambda x: x[1],reverse=True)
    # El índice 0 siempre corresponde a la misma película, por lo cual, no se considera
    score=score[1:6]
    return [i[0] for i in score]

In [12]:
df["index_movie"]=df.title.apply(index_movie)
df

,title,label,index_movie
0,toy story,toy stori anim comedi famili led woodi andi to...,"[17812, 2283, 1688, 8219, 13162]"
1,jumanji,jumanji adventur fantasi famili sibl judi pete...,"[13017, 12980, 20081, 4657, 5986]"
2,grumpier old men,grumpier old men romanc comedi famili wed reig...,"[2528, 1046, 11560, 1152, 11757]"
3,waiting to exhale,wait exhal comedi drama romanc cheat mistreat ...,"[1344, 10599, 6372, 1302, 519]"
4,father of the bride part ii,father bride part ii comedi just georg bank re...,"[5094, 7700, 18832, 9348, 7619]"
...,...,...,...
21801,the sublet,sublet mysteri drama thriller horror sublet su...,"[3294, 14645, 9843, 16550, 20204]"
21802,cadet kelly,cadet kelli comedi hyperact teenag kelli enrol...,"[3193, 21371, 1412, 7318, 17684]"
21803,pooh's heffalump halloween movie,pooh heffalump halloween movi anim famili hall...,"[13748, 721, 2926, 17247, 1516]"
21804,mom,mom crime drama thriller bliss biolog teacher ...,"[3946, 2760, 15756, 10395, 11753]"


### Exportación del df_movie para la realización del endpoint de ML (función 7)

In [16]:
# La columna "label" ya no será necesaria
df.drop(columns=["label"]).to_csv("API_ML_movie.csv",index=False)